In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("fixed_csv.csv", low_memory=False)


In [4]:
dropped_df = df.drop(labels=['JURISDICTION', 'AGE', 'PERSON TYPE'], axis=1)
encoded_df = pd.get_dummies(dropped_df, list(dropped_df.columns))
print(encoded_df.columns)

Index(['REASON FOR STOP_ACCIDENT', 'REASON FOR STOP_CALLS FOR SERVICE',
       'REASON FOR STOP_CHECK POINT', 'REASON FOR STOP_EQUIPMENT VIOLATION',
       'REASON FOR STOP_OTHER INVESTIGATIVE DETENTION',
       'REASON FOR STOP_TERRY STOP', 'REASON FOR STOP_TRAFFIC VIOLATION',
       'REASON FOR STOP_WARRANT SERVICE',
       'RACE_AMERICAN INDIAN OR ALASKA NATIVE',
       'RACE_ASIAN OR NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
       'RACE_BLACK OR AFRICAN AMERICAN', 'RACE_UNKNOWN', 'RACE_WHITE',
       'GENDER_FEMALE', 'GENDER_MALE', 'GENDER_OTHER', 'GENDER_UNKNOWN',
       'ENGLISH SPEAKING_NO', 'ENGLISH SPEAKING_YES', 'ACTION TAKEN_ARREST',
       'ACTION TAKEN_CITATION/SUMMONS', 'ACTION TAKEN_NO ENFORCEMENT ACTION',
       'ACTION TAKEN_WARNING ISSUED', 'PERSON SEARCHED_NO',
       'PERSON SEARCHED_YES', 'VEHICLE SEARCHED_NO', 'VEHICLE SEARCHED_YES',
       'FORCE USED BY OFFICER_NO', 'FORCE USED BY OFFICER_YES',
       'FORCE USED BY SUBJECT_NO', 'FORCE USED BY SUBJECT_YES'],


In [5]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(df=encoded_df, min_support=1e-3, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=1e-3)

In [17]:

desired_antecedents = {'RACE_AMERICAN INDIAN OR ALASKA NATIVE',
       'RACE_ASIAN OR NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
       'RACE_BLACK OR AFRICAN AMERICAN', 'RACE_WHITE', 'GENDER_MALE', 'GENDER_FEMALE'}
desired_consequents = {'ACTION TAKEN_ARREST'}
filtered_rules = rules[
    rules['consequents'].apply(lambda x: any(item in desired_consequents for item in x))
]
filtered_rules_race = filtered_rules[
    filtered_rules['antecedents'].apply(lambda x: len(x) == 1 and list(x)[0] in desired_antecedents)
]

In [22]:
desired_antecedents = {'REASON FOR STOP_ACCIDENT', 'REASON FOR STOP_CALLS FOR SERVICE',
       'REASON FOR STOP_CHECK POINT', 'REASON FOR STOP_EQUIPMENT VIOLATION',
       'REASON FOR STOP_OTHER INVESTIGATIVE DETENTION',
       'REASON FOR STOP_TERRY STOP', 'REASON FOR STOP_TRAFFIC VIOLATION',
       'REASON FOR STOP_WARRANT SERVICE'}
desired_consequents = {'ACTION TAKEN_ARREST'}
filtered_rules = rules[
    rules['consequents'].apply(lambda x: len(x) == 1 and list(x)[0] in desired_consequents)
]
filtered_rules_stop_reason = filtered_rules[
    filtered_rules['antecedents'].apply(lambda x: len(x) == 1 and list(x)[0] in desired_antecedents)
]

In [24]:
desired_antecedents = {'RACE_AMERICAN INDIAN OR ALASKA NATIVE',
       'RACE_ASIAN OR NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
       'RACE_BLACK OR AFRICAN AMERICAN', 'RACE_WHITE', 'GENDER_MALE', 'GENDER_FEMALE'}
desired_consequents = {'ACTION TAKEN_ARREST'}
filtered_rules = rules[
    rules['consequents'].apply(lambda x: len(x) == 1 and list(x)[0] in desired_consequents)
]
filtered_rules_race_gender = filtered_rules[
    filtered_rules['antecedents'].apply(lambda x: len(x) == 2 and list(x)[0] in desired_antecedents and list(x)[1] in desired_antecedents)
]

In [25]:
filtered_rules_race_gender.to_csv("filtered_rules_race_gender.csv", index=False)

In [23]:
filtered_rules_stop_reason.to_csv("filtered_rules_by_stop_reason.csv", index=False)

In [ ]:
filtered_rules_race.to_csv("filtered_rules_by_race_3.csv", index=False)

In [13]:
filtered_rules_race.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
222,(RACE_BLACK OR AFRICAN AMERICAN),(ACTION TAKEN_ARREST),0.298914,0.031471,0.013123,0.043901,1.39498,0.003716,1.013001,0.403864
270,(RACE_WHITE),(ACTION TAKEN_ARREST),0.632120,0.031471,0.017339,0.027429,0.87158,-0.002555,0.995845,-0.285977
